In [1]:
import pandas as pd
import numpy as np
seed = np.random.seed(42)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)

%matplotlib inline

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer,StandardScaler,LabelEncoder
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE

#f1_scorer = make_scorer(f1_score, average='macro',labels=Y.unique().tolist())
#myscorer = f1_scorer
myscorer = 'accuracy'

In [4]:
cdata = pd.read_csv('../data/central' +'_data.csv')
cdata.shape

(275, 73)

In [5]:
h1_p = pd.read_csv('../data/host1_predictions.csv')
h2_p = pd.read_csv('../data/host2_predictions.csv')
h3_p = pd.read_csv('../data/host3_predictions.csv')
h1_a = np.load('../data/host1_acc.npy')
h2_a = np.load('../data/host2_acc.npy')
h3_a = np.load('../data/host3_acc.npy')

In [6]:
h2_p.head(3)

,host2
0,1.0
1,1.0
2,0.0


In [7]:
data_e = cdata[cdata['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
cy_df = data_e['AFclass'].copy()
cx_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (270, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.002
[MICE] Starting imputation round 2/110, elapsed time 0.043
[MICE] Starting imputation round 3/110, elapsed time 0.066
[MICE] Starting imputation round 4/110, elapsed time 0.089
[MICE] Starting imputation round 5/110, elapsed time 0.111
[MICE] Starting imputation round 6/110, elapsed time 0.134
[MICE] Starting imputation round 7/110, elapsed time 0.157
[MICE] Starting imputation round 8/110, elapsed time 0.179
[MICE] Starting imputation round 9/110, elapsed time 0.202
[MICE] Starting imputation round 10/110, elapsed time 0.244
[MICE] Starting imputation round 11/110, elapsed time 0.267
[MICE] Starting imputation round 12/110, elapsed time 0.289
[MICE] Starting imputation round 13/110, elapsed time 0.312
[MICE] Starting imputation round 14/110, elapsed time 0.335
[MICE] Starting imputation round 15/110, elapsed time 0.358
[MICE] Starting imputation round 16/110, elapsed time 0.380
[MI

In [8]:
h1_a

array(0.68888889)

In [12]:
from sklearn.metrics import roc_auc_score,accuracy_score
input_p = pd.concat([h1_p, h2_p, h3_p],axis=1)
input_p.head(100)
iv = input_p.values
result = h1_a * iv[:,0] + h2_a * iv[:,1] + h3_a * iv[:,2] 
result /= (h1_a + h2_a + h3_a)
result[0:100]

array([0.65665548, 0.67258175, 0.34334452, 0.32741825, 0.        ,
       0.65665548, 0.32923724, 0.        , 0.        , 0.        ,
       0.        , 0.32923724, 0.        , 0.34334452, 0.        ,
       0.        , 0.        , 0.        , 0.34334452, 0.32923724,
       0.67258175, 0.        , 0.        , 0.32741825, 0.        ,
       0.        , 0.32923724, 0.34334452, 0.        , 0.67258175,
       0.65665548, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.34334452, 0.67258175, 0.        , 0.67258175,
       0.34334452, 0.67076276, 0.        , 0.34334452, 0.34334452,
       0.        , 0.        , 0.34334452, 0.32923724, 0.67076276,
       0.67258175, 0.67258175, 0.32923724, 0.34334452, 0.        ,
       0.65665548, 0.        , 0.32923724, 0.34334452, 0.        ,
       0.67258175, 0.34334452, 0.67076276, 0.        , 0.67076276,
       0.        , 0.        , 0.34334452, 0.32741825, 0.34334452,
       0.67076276, 0.67258175, 1.        , 0.        , 0.34334

In [14]:
result = np.where(result >= 0.5, 1, 0)
#roc_auc_score(cy_df, result)
accuracy_score(cy_df, result)

0.6666666666666666

In [16]:
param_grid = {"clf__max_depth": [10,30,50],
              "clf__criterion": ['gini', 'entropy'],
              "clf__n_estimators": [100,200,500,1000],
              "clf__random_state": [42],
              "clf__class_weight" : [None, 'balanced']}
# param_grid = {"classifier__max_depth": [30],
#               "classifier__n_estimators": [1000]}


rf = RandomForestClassifier()

plist = []
plist.append(('clf', rf))
pipeline = Pipeline(plist)
gs_rf = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [17]:
gs_rf.fit(input_p, cy_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   53.7s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.1min finished


In [18]:
gs_rf.best_params_

{'clf__class_weight': None,
 'clf__criterion': 'gini',
 'clf__max_depth': 10,
 'clf__n_estimators': 100,
 'clf__random_state': 42}

In [19]:
gs_rf.best_score_

0.674074074074074